# Steps to generate recommendations
1. Put your username and source in the cell below. For example, setting username="Fro116" and source="mal" will generate recommendations for the user "Fro116" on the site "myanimelist.net"
2. Run the script to generate a notebook of recommendations
3. View the recs by opening the notebook at the given path

In [ ]:
username = ""
source = ""

In [ ]:
assert username, "Must provide a username"
assert source in [
    "mal",
    "anilist",
    "kitsu",
    "animeplanet",
    "training",
], "Must provide a list source"

In [ ]:
import os
import shutil
import subprocess

import papermill as pm

In [ ]:
ALL_MEDIUMS = ["manga", "anime"]

In [ ]:
data_path = os.path.abspath("../../data/recommendations")
if not os.path.exists(data_path):
    os.mkdir(data_path)
if source == "training":
    rec_dir = os.path.join(data_path, username.split("@")[0], username.split("@")[1])
    papermill_path = os.path.join(rec_dir, "papermill")
else:
    rec_dir = os.path.join(data_path, source, str(username))
    papermill_path = os.path.join(rec_dir, "papermill", source)

In [ ]:
def run_julia_notebook(name, *args):
    os.system(f"julia {name} {' '.join(args)}")

## RecIngress

In [ ]:
def run_ingress_notebook(name, output_name=None, **kwargs):
    outdir = os.path.join(papermill_path, "rec_ingress")
    if not os.path.exists(outdir):
        os.makedirs(outdir, exist_ok=True)
    if output_name is None:
        output_name = name
    pm.execute_notebook(
        f"../RecIngress/{name}.ipynb",
        f"{outdir}/{name}.ipynb",
        parameters=dict(username=username, source=source, **kwargs),
    )

In [ ]:
def run_ingress_julia(name, *args):
    cwd = os.getcwd()
    os.chdir("../RecIngress")
    for folder in name.split("/")[:-1]:
        os.chdir(folder)
    try:
        run_julia_notebook(f"{name.split('/')[-1]}.jl", username, source, *args)
    finally:
        os.chdir(cwd)

In [ ]:
if os.path.exists(rec_dir):
    shutil.rmtree(rec_dir)

In [ ]:
%%time
run_ingress_notebook("FetchMediaLists")
original_source = source
username = str(username)
if source == "training":
    source, username = username.split("@")

In [ ]:
%%time
# TODO consolidate papermill startup costs
for medium in ALL_MEDIUMS:
    run_ingress_notebook("ImportMediaLists", f"Import{medium}Lists", media=medium)
    run_ingress_notebook("ProcessMediaLists", f"Process{medium}Lists", media=medium)
    run_ingress_notebook("GenerateMediaSplits", f"Generate{medium}Splits", media=medium)
temp_fn = "../ImportDatasets/gpu.lock"
if os.path.exists(temp_fn):
    os.remove(temp_fn)

In [ ]:
%%time
run_ingress_julia("CompressSplits")

## InferenceAlphas

In [ ]:
def run_inference_julia(name, *args):
    cwd = os.getcwd()
    os.chdir("../InferenceAlphas")
    for folder in name.split("/")[:-1]:
        os.chdir(folder)
    try:
        run_julia_notebook(f"{name.split('/')[-1]}.jl", username, source)
    finally:
        os.chdir(cwd)

In [ ]:
%%time
run_inference_julia("Baseline/Baseline")
run_inference_julia("Nondirectional/Nondirectional")
run_inference_julia("Transformer/Transformer")
run_inference_julia("BagOfWords/BagOfWords")
run_inference_julia("Ensemble/Ensemble")

## RecEgress

In [ ]:
def run_egress_notebook(name, output_name, **kwargs):
    outdir = os.path.join(papermill_path, "rec_egress")
    if not os.path.exists(outdir):
        os.makedirs(outdir, exist_ok=True)
    cwd = os.getcwd()
    os.chdir(os.path.dirname(name))
    try:
        pm.execute_notebook(
            f"{os.path.basename(name)}.ipynb",
            f"{outdir}/{output_name}.ipynb",
            parameters=dict(
                username=username,
                source=source,
                supress_injected_params=None,
                **kwargs,
            ),
        )
        subprocess.run(
            [
                "jupyter",
                "nbconvert",
                "--no-input",
                "--to",
                "html",
                f"{outdir}/{output_name}.ipynb",
            ],
            stderr=subprocess.DEVNULL,
        )
    finally:
        os.chdir(cwd)

In [ ]:
%%time
if original_source == "training":
    for medium in ALL_MEDIUMS:
        run_egress_notebook(
            f"./TrainingServingSkew",
            f"{medium.capitalize()}TrainingServingSkew",
            medium=medium,
        )

In [ ]:
%%time
run_egress_notebook("Recommendations/Recommendations", "Recommendations")

In [ ]:
output_fn = os.path.join(papermill_path, "rec_egress", "Recommendations.html")
print(f"Recommendations have been successfully written to {output_fn}")